# BIG DATA - Réseaux de neurones et Bagging avec Python
---

Ce travail a pour but de faire apprendre un classifier avec des reseau de neurones et du bagging et d'évaluer les performances de ce classifier sur des données de test.

### Imports et bibliothèques

In [1]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Perceptron 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

### 1 - Chargement de la base de données « Iris.txt »

In [2]:
data_iris = pd.read_csv("iris.txt", sep= "\t", header=None)
data_iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [3]:
# Transformation du DataFrame en un tableau numpy
data_array = data_iris.values

# Séparation des caractéristiques (X) de la variable à prédire (y)
X = data_array[:, :-1]
y = data_array[:, -1]

print(X.shape)
print(y.shape)

(150, 4)
(150,)


### 2 - Découpage de la base en Apprentissage/Test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=1)

print("Taille de l'ensemble d'apprentissage :", X_train.shape, y_train.shape)
print("Taille de l'ensemble de test :", X_test.shape, y_test.shape)

Taille de l'ensemble d'apprentissage : (100, 4) (100,)
Taille de l'ensemble de test : (50, 4) (50,)


### 3 - Implémentation d’un Perceptron Multi-classe

In [5]:
class PerceptronMultiClass:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

    def fit(self, X, y):
        unique_classes = np.unique(y)
        self.class_to_index = {c: i for i, c in enumerate(unique_classes)}
        self.index_to_class = {i: c for i, c in enumerate(unique_classes)}
        
        self.weights = np.zeros((X.shape[1], len(unique_classes)))
        self.bias = np.zeros(len(unique_classes))
        
        for _ in range(self.n_iterations):
            for i, x in enumerate(X):
                target_class_index = self.class_to_index[y[i]]
                
                for class_index in range(len(self.weights[0])):
                    target = 1 if class_index == target_class_index else 0
                    prediction = self.predict_one(x, class_index)
                    error = target - prediction
                    
                    self.weights[:, class_index] += self.learning_rate * error * x
                    self.bias[class_index] += self.learning_rate * error

    def predict_one(self, x, class_index):
        return np.dot(x, self.weights[:, class_index]) + self.bias[class_index]

    def predict(self, X):
        predictions = []
        for x in X:
            scores = [self.predict_one(x, class_index) for class_index in range(len(self.weights[0]))]
            predicted_class_index = np.argmax(scores)
            predicted_class = self.index_to_class[predicted_class_index]
            predictions.append(predicted_class)
        return np.array(predictions)

### 4 - Évaluation des performances du modèle

En premier lieu, il faut implémenter les fonctions qui font permettre d'evaluer les modèles.
Cela comprend : 
* Matrice de confusion
* Accuracy
* Precision
* Recall

In [6]:
def confusion_matrix(y_test, y_pred, num_classes):
    matrix = [[0] * num_classes for _ in range(num_classes)]
    for true, pred in zip(y_test, y_pred):
        if true <= num_classes and pred <= num_classes: 
            matrix[int(true)-1][int(pred)-1] += 1 
    return matrix


def precision_score(y_test, y_pred, num_classes):
    precision = []
    for i in range(1, num_classes+1):
        true_positive = sum(1 for true, pred in zip(y_test, y_pred) if true == i and pred == i)
        false_positive = sum(1 for true, pred in zip(y_test, y_pred) if true != i and pred == i)
        precision.append(true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0)
    return precision


def recall_score(y_test, y_pred, num_classes):
    recall = []
    for i in range(1, num_classes+1):
        true_positive = sum(1 for true, pred in zip(y_test, y_pred) if true == i and pred == i)
        false_negative = sum(1 for true, pred in zip(y_test, y_pred) if true == i and pred != i)
        recall.append(true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0)
    return recall


def accuracy_score(y_test, y_pred):
    correct = sum(1 for true, pred in zip(y_test, y_pred) if true == pred)
    return correct / len(y_test)


def evaluate_model(predictions, y_test, detailledMode=True):
    y_pred = predictions

    num_classes = len(set(y_test) | set(y_pred))

    conf_matrix = confusion_matrix(y_test, y_pred, num_classes)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, num_classes)
    recall = recall_score(y_test, y_pred, num_classes)

    if detailledMode:
        print("Matrice de confusion :")
        for row in conf_matrix:
            print(row)
        print("Accuracy globale :", accuracy)
        print("Précision pour chaque classe :", precision)
        print("Rappel pour chaque classe :", recall)
    return accuracy, precision, recall


On recherche les meilleurs parametres pour notre perceptron fait-maison.

Nous allons tester plusieurs hyper-parametres : 
*  `learning rate` : 0.1, 0.01, 0.001, 0.0001
* `iterations` : 1000, 2000

In [7]:
learning_rates = [0.1, 0.01, 0.001, 0.0001]
iterations = [1000, 2000]
best_accuracy = 0
best_combo = {'learning_rate': 0, 'iteration': 0}
results = []

for learning_rate in learning_rates:
    for iteration in iterations:
        perceptron = PerceptronMultiClass(learning_rate, iteration)
        perceptron.fit(X_train, y_train)
        predictions = perceptron.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        results.append({'learning_rate': learning_rate, 'iteration': iteration, 'accuracy': accuracy})
        if (accuracy > best_accuracy):
            best_accuracy = accuracy
            best_combo['learning_rate'] = learning_rate
            best_combo['iteration'] = iteration
        print("Learning rate :", learning_rate, " - Iterations :", iteration, " - Accuracy :", accuracy)

Learning rate : 0.1  - Iterations : 1000  - Accuracy : 0.34
Learning rate : 0.1  - Iterations : 2000  - Accuracy : 0.34
Learning rate : 0.01  - Iterations : 1000  - Accuracy : 0.64
Learning rate : 0.01  - Iterations : 2000  - Accuracy : 0.64
Learning rate : 0.001  - Iterations : 1000  - Accuracy : 0.74
Learning rate : 0.001  - Iterations : 2000  - Accuracy : 0.76
Learning rate : 0.0001  - Iterations : 1000  - Accuracy : 0.74
Learning rate : 0.0001  - Iterations : 2000  - Accuracy : 0.74



Finalement, nous observons que la combinaison de parametre qui maximise l'accuracy est un `learning rate` de 0.001 et un `nombre d'itérations` de 1000 (On s'aperçoit que cela ne change rien de changer le nombre d'itérations).

In [8]:
# Perceptron multi-classes fait maison avec les meilleurs paramètres
perceptron = PerceptronMultiClass(best_combo['learning_rate'], best_combo['iteration'])
perceptron.fit(X_train, y_train)
predictions = perceptron.predict(X_test)

print("Perceptron fait maison\n")
evaluate_model(predictions, y_test)


# Perceptron multi-classes de scikit-learn
perceptronScikit = Perceptron()
perceptronScikit.fit(X_train, y_train)
predictionsScikit = perceptronScikit.predict(X_test)


print("\n--------------------------------")
print("Perceptron de scikit-learn\n")
evaluate_model(predictionsScikit, y_test)

On s'aperçoit que le Perceptron de scikit-learn a une bien meilleure accuracy que celui que nous avons implémenté nous même.

### 5 - Implémentation d’un Percepron Multi-couches (MLP)

Nous allons essayer plusieurs paramètres pour le perceptron multi-couche afin de voir lesquels sont les meilleurs.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(3,), random_state=1)

params = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [1000, 2000]
}
# On utilise gridsearhc pour trouver les meilleurs paramètres
grid_search = GridSearchCV(estimator=mlp, param_grid=params, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres trouvés
print("Meilleurs paramètres trouvés:", grid_search.best_params_)
print("Meilleur score d'accuracy :", grid_search.best_score_)

Meilleurs paramètres trouvés: {'activation': 'tanh', 'learning_rate': 'constant', 'max_iter': 2000}
Meilleur score d'accuracy : 0.9699940582293524


### 6 - Évaluation des performances du modèle MLP

In [ ]:
# Utilisation des meilleurs paramètres trouvés
mlp = MLPClassifier(hidden_layer_sizes=(3,), activation=grid_search.best_params_['activation'], learning_rate=grid_search.best_params_['learning_rate'], max_iter=grid_search.best_params_['max_iter'], random_state=1)

mlp.fit(X_train, y_train)

predictionsMLP = mlp.predict(X_test)
evaluate_model(predictionsMLP, y_test)

Matrice de confusion :
[17, 0, 0]
[0, 18, 1]
[0, 0, 14]
Accuracy globale : 0.98
Précision pour chaque classe : [1.0, 1.0, 0.9333333333333333]
Rappel pour chaque classe : [1.0, 0.9473684210526315, 1.0]


(0.98, [1.0, 1.0, 0.9333333333333333], [1.0, 0.9473684210526315, 1.0])

Avec le MLP et les meilleurs parametres, on obtient une très bonne accuracy.

#### Normalisation des données

In [ ]:
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

mlp.fit(X_train_normalized, y_train)

predictions_mlp_normalized = mlp.predict(X_test_normalized)

evaluate_model(predictions_mlp_normalized, y_test)

Matrice de confusion :
[17, 0, 0]
[0, 18, 1]
[0, 1, 13]
Accuracy globale : 0.96
Précision pour chaque classe : [1.0, 0.9473684210526315, 0.9285714285714286]
Rappel pour chaque classe : [1.0, 0.9473684210526315, 0.9285714285714286]


(0.96,
 [1.0, 0.9473684210526315, 0.9285714285714286],
 [1.0, 0.9473684210526315, 0.9285714285714286])

#### Application avec les différents datasets

In [ ]:
def test_all_dataset(datasets, results, neurons):
    for dataset in datasets.keys():

        data = datasets[dataset]
        data_array = data.values
        
        X = data_array[:, :-1]
        y = data_array[:, -1]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
        
        # Perceptron multi classes fait maison
        perceptron = PerceptronMultiClass(learning_rate=0.001, n_iterations=1000)
        perceptron.fit(X_train, y_train)
        predictions = perceptron.predict(X_test)
            
        # MLP de scikit-learn    
        mlp = MLPClassifier(hidden_layer_sizes=(3,), random_state=1)
        mlp.fit(X_train, y_train)
        predictionsMLP = mlp.predict(X_test)
        
        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)
        
        # MLP de scikit-learn normalisée
        mlp.fit(X_train_normalized, y_train)
        predictions_mlp_normalized = mlp.predict(X_test_normalized)
        
        # On évalue les modèles
        accuracy1, precision1, rappel1 = evaluate_model(predictions, y_test, False)
        accuracy2, precision2, rappel2 = evaluate_model(predictionsMLP, y_test, False)
        accuracy3, precision3, rappel3 = evaluate_model(predictions_mlp_normalized, y_test, False)   
        
        # On ajoute les résultats au DataFrame
        results["Dataset"].append(dataset)
        results["Model"].append("Perceptron multi-classes fait maison")
        results["Accuracy"].append(str(round(accuracy1*100, 2)) + "%")
        results["Precision"].append(precision1)
        results["Rappel"].append(rappel1)
        
        results["Dataset"].append(dataset)
        results["Model"].append("Perceptron multi-couches de scikit-learn")
        results["Accuracy"].append(str(round(accuracy2*100, 2)) + "%")
        results["Precision"].append(precision2)
        results["Rappel"].append(rappel2)
        
        results["Dataset"].append(dataset)
        results["Model"].append("Perceptron multi-couches Scikit normalisé")
        results["Accuracy"].append(str(round(accuracy3*100, 2)) + "%")
        results["Precision"].append(precision3)
        results["Rappel"].append(rappel3)
    
    results_df = pd.DataFrame(results)
    results_df

    return results_df

In [ ]:
datasets = {
    "Glass": pd.read_csv("glass.txt", header=None, sep="\s+"),
    "Lsun": pd.read_csv("Lsun.txt", header=None, sep="\s+"),
    "Wave": pd.read_csv("Wave.txt", header=None, sep="\s+"),
    "Breast Cancer": pd.read_csv("breast-cancer-wisconsin.txt", header=None, sep="\s+")
}

results = {
    "Dataset": [],
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Rappel": []
}

results_df = test_all_dataset(datasets, results, 3)
results_df

,Dataset,Model,Accuracy,Precision,Rappel
0,Glass,Perceptron multi-classes fait maison,38.89%,"[0.3888888888888889, 0, 0, 0, 0, 0]","[1.0, 0.0, 0.0, 0, 0.0, 0.0]"
1,Glass,Perceptron multi-couches de scikit-learn,30.56%,"[0, 0.3055555555555556, 0, 0, 0, 0]","[0.0, 1.0, 0.0, 0, 0.0, 0.0]"
2,Glass,Perceptron multi-couches Scikit normalisé,11.11%,"[0, 0.3181818181818182, 0.0, 0, 0.020833333333...","[0.0, 0.3181818181818182, 0.0, 0, 0.3333333333..."
3,Lsun,Perceptron multi-classes fait maison,96.27%,"[0.9193548387096774, 1.0, 1.0]","[1.0, 0.8947368421052632, 0.9743589743589743]"
4,Lsun,Perceptron multi-couches de scikit-learn,29.1%,"[0, 0.0, 0.319672131147541]","[0.0, 0.0, 1.0]"
5,Lsun,Perceptron multi-couches Scikit normalisé,61.19%,"[0.8431372549019608, 0.0, 0.5492957746478874]","[0.7543859649122807, 0.0, 1.0]"
6,Wave,Perceptron multi-classes fait maison,79.42%,"[0.9144254278728606, 0.7723342939481268, 0]","[0.6702508960573477, 0.9387040280210157, 0]"
7,Wave,Perceptron multi-couches de scikit-learn,85.18%,"[0.8846153846153846, 0.8088012139605463, 0]","[0.8243727598566308, 0.9334500875656743, 0]"
8,Wave,Perceptron multi-couches Scikit normalisé,86.08%,"[0.8841121495327103, 0.8632478632478633, 0]","[0.8476702508960573, 0.8844133099824869, 0]"
9,Breast Cancer,Perceptron multi-classes fait maison,92.7%,"[0.9107142857142857, 0.9692307692307692]","[0.9870967741935484, 0.8076923076923077]"


#### Observations et hypothèses

Nous avons décidé de ne pas garder les hyper-parametres que nous avons trouvés précédemment car ces derniers ne sont optimisé que pour le dataset iris. Il aurait fallut refaire la demarche que nous avons fait précédemment de trouver les meilleurs paramètres pour chaque algo et pour chaque dataset.

Nous evaluerons les datasets uniquement en utilisant l'accuracy. Nous n'avons pas assez d'information sur les datasets et le contexte d'usage de ces derniers pour pouvoir choisir si la mesure la plus adaptée est le recall ou la précision (sauf pour le dataset breast-cancer-wisconsin).

Il a fallu modifier les fichiers contenant les datasets car les delimiters n'était pas constant à chaque ligne.

**Glass :**

Nombre de données : `214`

Nombre de classes : `6`

Pour ce dataset, on s'aperçoit que le Perceptron multi-classes fait-maison renvoie la meilleure accuracy. On remarque que cette accuracy n'est vraiment pas très élevée > `40 %`. De plus, on observe qu'avec le perceptron multi-couches de scikit-learn, on a une accuracy encore inférieure `30.56%` et d'autant plus inférieur quand les données sont normalisés. 

Les valeurs d'accuracy que l'on obtient ne nous permettent pas de choisir un algo en particulier. En effet, comme nous avons moins de 50% d'accuracy ce n'est pas très pertinent d'utiliser ces algos.

Une hypothèse pour expliquer cela serait les paramètres choisis qui ne sont pas adaptés. Plus particulièrement, on remarque que l'on a choisi 3 neurones pour le perceptron multi-couches. Cela peut sembler trop peu lorsque l'on regarde le dataset. En effet, les dataset propose 6 classes possibles (1,2,3,5,6,7). Ainsi, on pourrait peut-être améliorer l'accuracy en modifiant ce paramètre.
On remarque aussi que le nombre de données est très faibls comparé aux autres datasets. Augmenter le nombre de données pourrait permettre d'améliorer l'accuracy.

**Lsun :**

Nombre de données : `400`

Nombre de classes : `6`

Avec le perceptron multi-classes, on a une très bonne accuracy de `96.72%`. Le perceptron multi-couches sans normalisation renvoie une accuracy de `29.1%` et la normalisation augmente drastiquement l'accuracy à `61.19%`.

On prendrait donc comme algo pour entrainter notre modèle, le perceptron multi-classes.

Pareillement, il serait pertinent de trouver les hyper-paramètres qui maximisent l'accuracy pour avoir de meilleurs résultats avec le perceptron multi-couches.

In [ ]:
datasets = {
    "Glass": pd.read_csv("glass.txt", header=None, sep="\s+"),
}

results = {
    "Dataset": [],
    "Model": [],
    "Accuracy": []
    # "Precision": [],
    # "Rappel": []
}

results_df = test_all_dataset(datasets, results, 7)
results_df
 

,Dataset,Model,Accuracy
0,Glass,Perceptron multi-classes fait maison,38.89%
1,Glass,Perceptron multi-couches de scikit-learn,9.72%
2,Glass,Perceptron multi-couches Scikit normalisé,50.0%


On remarque donc ici qu'avec 7 couches, on a une bien meilleure accuracy soit `50%` avec un nombre de neurones de 7.

**Wave :**

Nombre de données : `5000`

Nombre de classes : `6`

On a ici, un très bonne accuracy pour chacun des algos utilisés. On remarque que le perceptron multi-couches est meilleur que notre perceptron multi-classes et que la normalisation améliore encore légérement l'accuracy pour atteindre `86.08%`.

On prendrait donc comme algo pour entrainer notre modèle, le perceptron multi-couches avec les données normalisées.

Pareillement, il serait pertinent de trouver les hyper-paramètres qui maximisent l'accuracy pour avoir de meilleurs résultats avec le perceptron multi-couches.

**Breast Cancer :**

Nombre de données : `699`

Nombre de classes : `2`

Les classifiers ont atteint les meilleurs scores avec ce dataset. En effet, on peut voir que le perceptron multi-classes renvoit une accuracy de `92.7%`. Le perceptron multi-couches renvoit une accuracy de `83.69%` et de `95.71%`.

On prendrait donc comme algo pour entrainer notre modèle, le perceptron multi-couches avec les données normalisées.

Pour ce jeu de données ont pourrait en plus utiliser une autre mesure pour évaluer le modèle soit le recall. En effet, ce dataset concerne des potentiels malades du cancer du sein au wisconsin. On pourrait imaginer que le contexte d'utilisation du modèle serait de determiner si une personne est atteinte au non du cancer du sein. On veut donc limiter le plus possible les faux negatifs soit le nombre de mauvais diagnostics (une personne n'est pas malade alors qu'elle l'est vraiment). Il faut ainsi maximiser le recall. 
On remarque que c'est le perceptron multi-couches avec les données normalisés. En effet, on est à : `93.5%`

**Conclusion :**

On remarque que en général, la normalisation des données améliore le client
Les mauvaises performances du MLP peuvent s'expliquer par un paramètrages non approprié au problèmes et au dataset.

Pour obtenir des meilleurs résultats, il aurait fallu tester plusieurs combinaisons de paramètres pour chaque dataset pour trouver des paramètres qui maximisent l'accuracy.

### 7 - Bagging de réseaux de neurones

In [ ]:
# Fonction de création d'échantillons bootstrap
def create_bootstrap_samples(X_train, y_train, K):
    bootstrap_samples = []
    for _ in range(K):
        X_bootstrap, y_bootstrap = resample(X_train, y_train)
        bootstrap_samples.append((X_bootstrap, y_bootstrap))
    return bootstrap_samples

# Fonction qui permet de rééchantillonner les données
def resample(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]

# Fonction qui permet d'entraîner un classifieur MLP
def train_mlp_classifier(X_train, y_train):
    mlp = MLPClassifier(hidden_layer_sizes=(100,), random_state=1)
    mlp.fit(X_train, y_train)
    return mlp

# Fonction qui permet d'agréger les prédictions des classifieurs
def aggregate_models(classifiers):
    def predict_ensemble(X):
        predictions = np.array([clf.predict(X) for clf in classifiers])
        aggregated_predictions = np.mean(predictions, axis=0)
        return np.round(aggregated_predictions).astype(int)
    return predict_ensemble


In [ ]:
K = int(input("Entrez le nombre d'échantillons bootstrap (K) : "))
for dataset in datasets.keys():
    # On teste sur le dataset Iris
    data = datasets[dataset]

    data_array = data.values
    X = data_array[:, :-1]
    y = data_array[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
    
    # Création des échantillons
    bootstrap_samples = create_bootstrap_samples(X_train, y_train, K)
    # On créer les classifieurs pour chaque échantillon
    classifiers = [train_mlp_classifier(X_bootstrap, y_bootstrap) for X_bootstrap, y_bootstrap in bootstrap_samples]
    # On agrège les prédictions des classifieurs
    ensemble_classifier = aggregate_models(classifiers)
    # On fait les prédictions sur l'ensemble de test
    predictions_ensemble = ensemble_classifier(X_test)

    print("\n------------------")
    print("Dataset :", dataset)
    evaluate_model(y_test, predictions_ensemble)


------------------
Dataset : Glass
Matrice de confusion :
[0, 0, 0, 0, 0, 0]
[28, 22, 7, 0, 3, 2]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
Accuracy globale : 0.3055555555555556
Précision pour chaque classe : [0.0, 1.0, 0.0, 0, 0.0, 0.0]
Rappel pour chaque classe : [0, 0.3055555555555556, 0, 0, 0, 0]

------------------
Dataset : Lsun
Matrice de confusion :
[57, 0, 0]
[0, 38, 0]
[0, 0, 39]
Accuracy globale : 1.0
Précision pour chaque classe : [1.0, 1.0, 1.0]
Rappel pour chaque classe : [1.0, 1.0, 1.0]

------------------
Dataset : Wave
Matrice de confusion :
[449, 39, 62]
[42, 490, 31]
[67, 42, 445]
Accuracy globale : 0.8302339532093581
Précision pour chaque classe : [0.8046594982078853, 0.8581436077057794, 0]
Rappel pour chaque classe : [0.8163636363636364, 0.8703374777975134, 0]

------------------
Dataset : Breast Cancer
Matrice de confusion :
[152, 7]
[3, 71]
Accuracy globale : 0.9570815450643777
Précision pour chaque classe : [0.9806451612903225,

En testant avec K = 5, on remarque que les resultats sont bien meilleurs pour le dataset Glass et Lsun avec le perceptron multi-couches.
Pour le dataset Wave, les résultats sont similaires avec ou sans bagging.
Pour le dataset Breast Cancer, les résultats sont meilleurs pour le perceptron multi-couches normalisé sans le bagging.